데이터의 특성과 분포를 유지한채 값을 계산하기 편하게 스케일링 하는것을 정규화라 한다
값이 너무 커질경우 컴퓨터가 계산을 하지 못하기 때문에 해준다

1. max > 0
   max - min > 0
2. max = 0
   max - min > 0
3. max < 0
   max - min > 0
    
데이터의 최댓값과 최솟값을 구해서 나눠주는 방식으로 정규화를 한다

0< data-min/ max-min < 1 
    
TDD방식의 개발법
기능검증을 위해 노드 한개만을 제공한 후 돌아가는 방식을 체크한다

가중치가 많아지면 최적화가 하기 힘들어진다

이번 예제의 목적1. Mnist class 구조 파악
2. 정규화와 one-hot encoding best fit
3. 문제점 도출

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [6]:
# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [7]:
class MNist:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, name, input_nodes, hidden1_nodes, output_nodes, learning_rate):
        
        self.name = name
        
        # 2층 hidden layer unit 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden1_nodes)  
        self.b2 = np.random.rand(hidden1_nodes)
        
        # 3층 hidden layer unit  
        self.W3 = np.random.rand(hidden1_nodes, output_nodes)
        self.b3 = np.random.rand(output_nodes)
        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        print(self.name, " is created !!!")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
            
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z2 = np.dot(input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
        
        # one-hot encoding 방식으로 하기 위한 코드 수정
        predicted_num = np.argmax(y)
    
        return predicted_num
    
    def accuracy(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(input_data)):
            
            # one-hot encoding 방식으로 하기 위한 코드 수정
            label = int(target_data[index])
            
            predicted_num = self.predict(input_data[index])
            
            if predicted_num == label:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        #print("Accuracy => ", accuracy_result)
        
        return not_matched_list, accuracy_result
        
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)

In [8]:
class DataGeneration:
    
    # target_position = 0 (첫번째열이 정답데이터), target_position=-1 (마지막열이 정답데이터)
    def __init__(self, name, file_path, seperation_rate, target_position=-1):
        
        self.name = name
        
        self.file_path = file_path
        
        self.seperation_rate = seperation_rate
        
        if (target_position == -1  or  target_position == 0):      
            self.target_position = target_position
        
        else:
            err_str = 'target_position must be -1 or 0'            
            raise Exception(err_str)    
            
    
    # print data target distribution 
    # str_of_kind : 'original data' or  'training data'  or  'test data'
    def __display_target_distribution(self, data, str_of_kind='original data'):
        
        print('=======================================================================================================')
        
        target_data = data[ :, self.target_position ]
        
        # numpy.unique() 사용하여 loaded data target 분포 확인
        unique, counts = np.unique(target_data, return_counts=True)

        unique_target = []
    
        for index in range(len(unique)):
        
            print('[DataGeneration] unique number of ' + str_of_kind + ' = ', unique[index], ', count = ', counts[index])
        
            unique_target.append(unique[index])

        for index in range(len(unique_target)):
        
            print('[DataGeneration] unique number of ' + str_of_kind + ' = ', unique_target[index], ', ratio = ', np.round(100 * counts[index] / (target_data.shape[0]), 2), ' %')
    
        print('=======================================================================================================')
        
        
    # numpy.random.shuffle()  이용하여 training_data / test_data 생성
    def generate(self):
    
        # 데이터 불러오기, 파일이 없는 경우 exception 발생

        try:
            loaded_data = np.loadtxt(self.file_path, delimiter=',', dtype=np.float32)
            
        except Exception as err:
            print('[DataGeneration::generate()]  ', str(err))
            raise Exception(str(err))

        print("[DataGeneration]  loaded_data.shape = ", loaded_data.shape)
            
        # print the target distribution of original data 
        
        self.__display_target_distribution(loaded_data, 'original data')
        
        
        # 분리비율에 맞게 테스트데이터로 분리
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)

        # numpy.random.shuffle 을 이용하여 랜덤하게 데이터 섞기
        np.random.shuffle(loaded_data)
        
        # test_data 는 0 : test_data_num
        
        
        test_data = loaded_data[ 0:test_data_num ]

        # training_data 는 test_data_num 부터 끝까지 
        training_data = loaded_data[ test_data_num: ]

        # display target distribution of generated data 
        
        self.__display_target_distribution(training_data, 'training data')
        
        self.__display_target_distribution(test_data, 'test data')
        
        return training_data, test_data

In [13]:
training_data = np.loadtxt('./mnist_train.csv', delimiter=',', dtype=np.float32)

print("training_data.shape = ", training_data.shape)

training_input_data = training_data[ : , 1: ]
training_target_data = training_data[ : , 0 ]

print("training_input_data.shape = ", training_input_data.shape, ', training_target_data.shape = ', training_target_data.shape)

test_data = np.loadtxt('./mnist_test.csv', delimiter=',', dtype=np.float32)

print("test_data.shape = ", test_data.shape)

test_input_data = test_data[ : , 1: ]
test_target_data = test_data[ : , 0 ]

print("test_input_data.shape = ", test_input_data.shape, ', test_target_data.shape = ', test_target_data.shape)

training_data.shape =  (60000, 785)
training_input_data.shape =  (60000, 784) , training_target_data.shape =  (60000,)
test_data.shape =  (10000, 785)
test_input_data.shape =  (10000, 784) , test_target_data.shape =  (10000,)


In [14]:
# max / min
max_input_val = np.max(training_input_data)
max_target_val = np.max(training_target_data)

min_input_val = np.min(training_input_data)
min_target_val = np.min(training_target_data)



print('max_input_val = ', max_input_val, ', max_target_val = ', max_target_val)
print('min_input_val = ', min_input_val, ', min_target_val = ', min_target_val)

max_input_val =  255.0 , max_target_val =  9.0
min_input_val =  0.0 , min_target_val =  0.0


In [15]:
each_max_val = np.max(training_input_data, axis=1)    # 각 행의 최대값
each_min_val = np.min(training_input_data, axis=1)    # 각 행의 최소값

print('each_max_val = ', each_max_val, ', len(each_max_val) = ', len(each_max_val))
print('each_min_val = ', each_min_val, ', len(each_min_val) = ', len(each_min_val))

# axis = 0 지정하면 각 열의 최대 값과 최소 값을 구할 수 있다

each_max_val =  [255. 255. 255. ... 255. 255. 254.] , len(each_max_val) =  10000
each_min_val =  [0. 0. 0. ... 0. 0. 0.] , len(each_min_val) =  10000


In [23]:
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 1  # hidden 1 nodes 개수. 

o_nodes = 2    # one-hot encoding 표현하기 위한 output nodes 개수는 2개. 즉 정답은 0 또는 1 두개만 있음

lr = 1e-2      # learning rate. hi_node = 2, 1e-1 에서 수렴
epochs = 20   # 반복횟수. 

# 저장 리스트
loss_val_list = []    # loss val list

# Diabetes 객체 생성
obj1 = MNist("Mnist", i_nodes, h1_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        
        # one-hot encoding 방식으로 표현                
        target_data = np.zeros(o_nodes) + 0.01    
        target_data[int(training_data[index, -1])] = 0.99
        # 0으로 초기화 후 원핫 인코딩 방식으로 표현
        
        obj1.train(input_data, target_data)
        
    cur_loss_val = obj1.loss_val()    
    loss_val_list.append(cur_loss_val)
    
    print("step = ", step, " , current loss value = ", cur_loss_val)

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

Mnist  is created !!!
Neural Network Learning using Numerical Derivative...
step =  0  , current loss value =  0.11200266932457137


KeyboardInterrupt: 